In [1]:
#Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering, KMeans, AffinityPropagation
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from utils import *

In [2]:
# Load the dataset
biomass_data = pd.read_csv('site_data.csv')

# Transpose the DataFrame to have products as rows and locations as columns
data_transposed = biomass_data.set_index(['dataset', 'site_name', 'site_id', 'X', 'Y']).T

# Remove non-numeric rows
data_transposed_numeric = data_transposed.select_dtypes(include=[np.number])

# Replace 'NA' with NaN
data_transposed_numeric = data_transposed_numeric.replace('NA', np.nan)

# Drop rows with all NaN values
data_transposed_numeric = data_transposed_numeric.dropna(axis=1, how='all')

# Impute missing values with column mean
imputer = SimpleImputer(strategy='mean')
# Impute missing values with ROW mean (must use transposed df and axis=1 for rows)
data_imputed = data_transposed_numeric.T.fillna(data_transposed_numeric.mean(axis=1)).T

data_imputed

dataset                    NEON                                             \
site_name        Harvard Forest Smithsonian Conservation Biology Institute   
site_id                    HARV                                       SCBI   
X                   -72.172660                                 -78.139500    
Y                     42.536900                                  38.892920   
ESA CCI               21.046901                                  21.046901   
Chopping et al.       18.783055                                  18.783055   
GEDI L4B             164.080719                                 334.279755   
Liu et al.            37.921211                                 115.212837   
LT-GNN                31.403246                                  31.403246   
Menlove & Healey     114.922806                                 106.782768   
Xu et al.             69.325028                                  69.325028   

dataset                                                         \
site_name        Ordway-Swisher Biological Station      UNDERC   
site_id                                       OSBS        UNDE   
X                                      -81.993430  -89.537250    
Y                                        29.689270   46.233880   
ESA CCI                                  21.046901   21.046901   
Chopping et al.                          18.783055   18.783055   
GEDI L4B                                 63.379155  155.069643   
Liu et al.                               69.890287  139.331772   
LT-GNN                                   31.403246   31.403246   
Menlove & Healey                         42.734001   91.699257   
Xu et al.                                69.325028   69.325028   

dataset                                                        \
site_name        Konza Prairie Biological Station   Oak Ridge   
site_id                                      KONZ        ORNL   
X                                     -96.563090  -84.282600    
Y                                       39.100770   35.964120   
ESA CCI                                 21.046901   21.046901   
Chopping et al.                         18.783055   18.783055   
GEDI L4B                                 7.477551  162.482702   
Liu et al.                              10.838812   91.845345   
LT-GNN                                  31.403246   31.403246   
Menlove & Healey                        10.135107  115.705890   
Xu et al.                               69.325028   69.325028   

dataset                                                 \
site_name        Talladega National Forest   Woodworth   
site_id                               TALL        WOOD   
X                              -87.393270  -99.241356    
Y                                32.950460   47.128228   
ESA CCI                          21.046901   21.046901   
Chopping et al.                  18.783055   18.783055   
GEDI L4B                        228.355491    4.161302   
Liu et al.                       85.784576    9.719982   
LT-GNN                           31.403246   31.403246   
Menlove & Healey                125.903267    0.000000   
Xu et al.                        69.325028   69.325028   

dataset                                                                    \
site_name        Central Plains Experimental Range LBJ National Grassland   
site_id                                       CPER                   CLBJ   
X                                      -104.745602            -97.570000    
Y                                        40.815534              33.401230   
ESA CCI                                  21.046901              21.046901   
Chopping et al.                           0.000000              18.783055   
GEDI L4B                                  1.268689              17.868778   
Liu et al.                                9.621728              11.764260   
LT-GNN                                   31.403246              31.403246   
Menlove & Healey      

In [4]:
df=data_imputed

In [ ]:
#Test 1: make 10 datasets based on Liu. With a noise scaling factor

# Specify the mean and standard deviation of the Gaussian noise
mean = np.mean(df.loc['Liu et al.'])
std_dev = np.std(df.loc['Liu et al.'])
var = np.var(df.loc['Liu et al.'])

# Generate random noise from a Gaussian distribution
noise = np.random.normal(mean, std_dev, df.loc['Liu et al.'].shape)

# Add the noise to the dataset
noisy_data_1 = df.loc['Liu et al.'] + noise   

In [ ]:
mean

In [ ]:
# Initialize the noise scale factor
noise_scale = 1.0

# Generate random noise and add it to the original data 10 times
for i in range(1, 11):
    # Scale the standard deviation based on the loop iteration
    scaled_std_dev = std_dev * noise_scale
    
    # Generate random noise with the scaled standard deviation
    noise = np.random.normal(mean, scaled_std_dev, df.loc['Liu et al.'].shape)
    
    # Add the noise to the original data
    noisy_data_Liu = df.loc['Liu et al.'] + noise

    # Construct the variable name and assign the noisy data
    var_name = f'noisy_data_{i}'
    globals()[var_name] = noisy_data
    
    # Increase the noise scale factor for the next iteration
    noise_scale += 0.5

In [ ]:
n = 10  # Change this to the number of noisy data vectors you have

# Create an empty list to store the noisy data vectors
noisy_data_list = []

# Append each noisy data vector to the list
for i in range(1, n+1):
    var_name = f'noisy_data_{i}'
    noisy_data_vector = globals()[var_name]
    noisy_data_list.append(noisy_data_vector)

# Stack the noisy data vectors along the vertical axis to form the new dataset
noisy_data_dataset = np.vstack(noisy_data_list)
noisy_data_dataset

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(noisy_data_dataset)

cluster = AgglomerativeClustering(n_clusters=2, linkage='ward')
clusters = cluster.fit_predict(data_scaled)

if -1 in clusters:
    print("noise found during clustering, contained in cluster -1")

In [ ]:
clusters

In [5]:
def generate_noisy_data(original_data, n_vectors):
    """
    Generate n_vectors of noisy data from the given original_data.
    
    Args:
        original_data (numpy.ndarray): The original data array.
        n_vectors (int): The number of noisy data vectors to generate.
        
    Returns:
        numpy.ndarray: A 2D array containing the n_vectors of noisy data,
                        with each row representing one noisy data vector.
    """
    mean = np.mean(original_data)
    std_dev = np.std(original_data)
    
    noisy_data_list = []
    for _ in range(n_vectors):
        noise = np.random.normal(mean, std_dev, original_data.shape)
        noisy_data = original_data + noise
        noisy_data_list.append(noisy_data)
    
    return np.vstack(noisy_data_list)

# Generate 10 vectors of noisy data from the first dataset
noisy_data_1 = generate_noisy_data(df.loc['Liu et al.'], 10)

# Generate 10 vectors of noisy data from the second dataset
noisy_data_2 = generate_noisy_data(df.loc['GEDI L4B'], 10)

# Combine the noisy data vectors from both datasets
Liu_Gedi = np.vstack((noisy_data_1, noisy_data_2))
Liu_Gedi

array([[139.30267657, 232.96131899, 119.04963722, ..., 206.62545021,
        213.68517957,  73.15109124],
       [ 82.93466021, 153.40668196, 124.18889895, ..., 262.65649041,
          3.36904635,  86.69594518],
       [ 33.87134755,  84.2327844 , 121.46424622, ..., 180.147882  ,
        100.94698919,  44.34349035],
       ...,
       [349.52722685, 487.93017236,  88.39186357, ..., 402.1479478 ,
         66.92332817, -60.98227256],
       [177.48737548, 352.67430687, 167.61578434, ..., 360.48319859,
        -24.87188334,  23.12093094],
       [145.69148822, 307.07878323,  64.21689124, ..., 441.15198891,
         23.4367843 , 209.57005299]])

In [6]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(Liu_Gedi)

cluster = AgglomerativeClustering(n_clusters=2, linkage='ward')
clusters = cluster.fit_predict(data_scaled)

if -1 in clusters:
    print("noise found during clustering, contained in cluster -1")

In [7]:
clusters

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)